# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Требуется построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

Задача: на основе данных о поведении клиентов, которые уже перешли на эти тарифы, построить модель для задачи классификации, которая выберет подходящий тариф.

Цель: постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

Подгрузим необходимые для выполнения задания библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

random_st = 42 # определим начальное состояние генератора случайных чисел, будем использовать его в дальнейшем

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# выведем первые 5 объектов
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
# посмотрим общую информацию по данным
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Вывод:

1. Исходные данные состоят из 3214 объектов с 5 признаками.
2. Предобработка данных не требуется.

## 2. Разбейте данные на выборки

Разделим данные на обучающую, тестовую и валидационную выборки. Выберем следующие пропорции выборок (3-1-1):
- 60% - обучающий набор;
- 20% - валидационная выборка;
- 20% - тестовая выборка.

In [5]:
# поделим данные на обучающий набор и тестовую+валидационную выборку, задав состояние random_state
data_train, data_testval = train_test_split(data, test_size=0.4, random_state=random_st)

In [6]:
# поделим тестовую+валидационную выборку на отдельные тестовые и валидационную выборку
data_test, data_valid = train_test_split(data_testval, test_size=0.5, random_state=random_st)

Проверим себя, посмотрим на размерность получившихся датафреймов:

In [7]:
for i in ((data_train, 'Данные для тренеровки модели'), 
          (data_test, 'Данные для тестирования модели'), 
          (data_valid, 'Данные для настройки и проверки модели')):
    print('{} имеют размерность => {} и составляют {:.1%} от исходных данных'.format(i[1], 
                                                                                 i[0].shape, 
                                                                                 (i[0].shape[0]/data.shape[0])))

Данные для тренеровки модели имеют размерность => (1928, 5) и составляют 60.0% от исходных данных
Данные для тестирования модели имеют размерность => (643, 5) и составляют 20.0% от исходных данных
Данные для настройки и проверки модели имеют размерность => (643, 5) и составляют 20.0% от исходных данных


### Вывод:


Данные разделены на обучающую, тестовую и валидационную выборки в следующей пропорции - 60%-20%-20%

## 3. Исследуйте модели

Для обучения моделей классификации используем три популярных алгоритма, сравним их и выберем тот, который более всего подходит по параметру accuracy.

Для начала выделим из ранее сделанных выборок датафрейм с признаками и целевой признак:

In [8]:
# определим функцию, которая будет возвращать объекты целевого признака и остальных признаков
def get_features(data):
    features = data.drop(['is_ultra'], axis=1)
    target = data['is_ultra']
    return (features, target)

In [9]:
# данные для тренеровки
features_train, target_train = get_features(data_train)

# данные для проверки модели
features_valid, target_valid = get_features(data_valid)

# даные для тестирования модели
features_test, target_test = get_features(data_test)

Данные готовы для обучения и проверок моделей! Выберем алгоритмы обучения моделей:
- DecisionTreeClassifier;
- RandomForestClassifier;
- LogisticRegression.

Запишем алгоритмы классификации в соответствующие переменные, установим некоторые первоначальные параметры и обучим модели:

In [10]:
# для дерева решений выберем в первом приближении максимальную глубину дерева = 3 (количество переходов между узлами для принятия решения)
model_dtc = DecisionTreeClassifier(max_depth=3, random_state=random_st)

# для случайного леса выберем количество деревьев в лесе = 2 и максимальную глубину дерева = 3
model_rfc = RandomForestClassifier(n_estimators=2, max_depth=3, random_state=random_st)

# для модели логистической регрессии не будем определять никаких гиперпараметров
model_lr = LogisticRegression(random_state=random_st)

Обучим сразу все модели на данных для обучения:

In [11]:
for model in (model_dtc, model_rfc, model_lr):
    model.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Модели обучены. Нужно оценить, насколько точны предсказания каждой из них:

In [12]:
for model in (model_dtc, model_rfc, model_lr):
    print('Качество модели (accuracy) равно {:.1%}'.format(model.score(features_train, target_train)))

Качество модели (accuracy) равна 79.7%
Качество модели (accuracy) равна 77.6%
Качество модели (accuracy) равна 71.4%


Чем выше качество модели, тем лучше. В первом приближении лучше дерево решений. Попробуем сделать предсказание на валидационной выборке и оценить качество полученных результатов:

In [13]:
# определим функцию для автоматизации предсказаний и вычисления качества этих предсказаний
def get_accuracy(model, features=features_valid, target=target_valid):
    predictions = model.predict(features)
    return accuracy_score(target, predictions)

In [14]:
# вычислим точность предсказаний каждой модели
accuracy_dtc = get_accuracy(model_dtc)
accuracy_rfc = get_accuracy(model_rfc)
accuracy_lr = get_accuracy(model_lr)

def print_accuracy(ac1, ac2, ac3):
    for accuracy in ((ac1, 'DecisionTreeClassifier'), 
                     (ac2, 'RandomForestClassifier'), 
                     (ac3, 'LogisticRegression')):
        print('Качество предсказаний модели {} => {:.2%}'.format(accuracy[1], accuracy[0]))
        
print_accuracy(accuracy_dtc, accuracy_rfc, accuracy_lr)

Качество предсказаний модели DecisionTreeClassifier => 80.56%
Качество предсказаний модели RandomForestClassifier => 76.98%
Качество предсказаний модели LogisticRegression => 70.92%


Качество на валидационной выборке удовлетворяет для первых двух алгоритмов, но посмотрим, нельзя ли их еще улучшить.

#### 3.1 Улучшение модели случайного леса

Можно попробовать улучшить модель случайного леса методом подбора гиперпараметров. Попробуем увеличить количество деревьев и оценить изменения в качестве модели.

In [15]:
for n_estim in range(1, 20):
    model_rfc_new = RandomForestClassifier(n_estimators=n_estim, max_depth=3, random_state=random_st)
    model_rfc_new.fit(features_train, target_train)
    predictions_valid = model_rfc_new.predict(features_valid)
    accuracy_valid = accuracy_score(target_valid, predictions_valid)
    print('Качество предсказаний случайного леса с количеством деревьев = {} => {:.2%}'.format(n_estim, accuracy_valid))

Качество предсказаний случайного леса с количеством деревьев = 1 => 75.12%
Качество предсказаний случайного леса с количеством деревьев = 2 => 76.98%
Качество предсказаний случайного леса с количеством деревьев = 3 => 81.65%
Качество предсказаний случайного леса с количеством деревьев = 4 => 79.78%
Качество предсказаний случайного леса с количеством деревьев = 5 => 79.94%
Качество предсказаний случайного леса с количеством деревьев = 6 => 79.63%
Качество предсказаний случайного леса с количеством деревьев = 7 => 80.72%
Качество предсказаний случайного леса с количеством деревьев = 8 => 79.63%
Качество предсказаний случайного леса с количеством деревьев = 9 => 80.56%
Качество предсказаний случайного леса с количеством деревьев = 10 => 80.87%
Качество предсказаний случайного леса с количеством деревьев = 11 => 81.03%
Качество предсказаний случайного леса с количеством деревьев = 12 => 80.56%
Качество предсказаний случайного леса с количеством деревьев = 13 => 80.72%
Качество предсказаний

Из расчета видно, что качество модели с количеством деревьев = 3 лучше, чем при 2 на 5% и лучше дерева решений на 1%. Попробуем подкрутить глубину max_depth.

In [16]:
for depth in range(3, 20):
    model_rfc_new = RandomForestClassifier(n_estimators=3, max_depth=depth, random_state=random_st)
    model_rfc_new.fit(features_train, target_train)
    predictions_valid = model_rfc_new.predict(features_valid)
    accuracy_valid = accuracy_score(target_valid, predictions_valid)
    print('Качество предсказаний случайного леса с глубиной = {} => {:.2%}'.format(depth, accuracy_valid))

Качество предсказаний случайного леса с глубиной = 3 => 81.65%
Качество предсказаний случайного леса с глубиной = 4 => 80.25%
Качество предсказаний случайного леса с глубиной = 5 => 81.80%
Качество предсказаний случайного леса с глубиной = 6 => 81.34%
Качество предсказаний случайного леса с глубиной = 7 => 81.18%
Качество предсказаний случайного леса с глубиной = 8 => 79.47%
Качество предсказаний случайного леса с глубиной = 9 => 79.94%
Качество предсказаний случайного леса с глубиной = 10 => 80.40%
Качество предсказаний случайного леса с глубиной = 11 => 79.78%
Качество предсказаний случайного леса с глубиной = 12 => 78.07%
Качество предсказаний случайного леса с глубиной = 13 => 78.69%
Качество предсказаний случайного леса с глубиной = 14 => 81.03%
Качество предсказаний случайного леса с глубиной = 15 => 78.69%
Качество предсказаний случайного леса с глубиной = 16 => 78.38%
Качество предсказаний случайного леса с глубиной = 17 => 79.94%
Качество предсказаний случайного леса с глубино

При глубине дерева 5 модель немного улучшается (81.8%). Выберем параметры для предсказаний на тестовой выборке - n_estims=3 и max_depth=5.

In [17]:
model_rfc = RandomForestClassifier(n_estimators=3, max_depth=5, random_state=random_st)

# обучим модель
model_rfc.fit(features_train, target_train);

#### 3.2 Улучшение модели дерева решений

Попробуем улучшить модель дерева решений, изменив гиперпараметр max_depth.

In [18]:
for depth in range(1, 10):
    model_dtc_new = DecisionTreeClassifier(max_depth=depth, random_state=random_st)
    model_dtc_new.fit(features_train, target_train)
    predictions_valid = model_dtc_new.predict(features_valid)
    accuracy_valid = accuracy_score(target_valid, predictions_valid)
    print('Качество предсказаний дерева решений с глубиной = {} => {:.2%}'.format(depth, accuracy_valid))

Качество предсказаний дерева решений с глубиной = 1 => 74.34%
Качество предсказаний дерева решений с глубиной = 2 => 79.16%
Качество предсказаний дерева решений с глубиной = 3 => 80.56%
Качество предсказаний дерева решений с глубиной = 4 => 81.18%
Качество предсказаний дерева решений с глубиной = 5 => 80.72%
Качество предсказаний дерева решений с глубиной = 6 => 80.09%
Качество предсказаний дерева решений с глубиной = 7 => 79.78%
Качество предсказаний дерева решений с глубиной = 8 => 79.78%
Качество предсказаний дерева решений с глубиной = 9 => 79.63%


При глубине дерева = 4 качество модели улучшается на чуть менее 1%. Переобучим модель дерева решений с другой глубиной:

In [19]:
model_dtc = DecisionTreeClassifier(max_depth=4, random_state=random_st)

#обучим модель
model_dtc.fit(features_train, target_train);

#### 3.3 Улучшение модели логистической регрессии

Для улучшения предсказаний модели логистической регрессии изменим алгоритм оптимизации.

In [20]:
for alg in ('newton-cg', 'liblinear', 'sag', 'saga'):
    model_lr_new = LogisticRegression(random_state=random_st, solver=alg)
    model_lr_new.fit(features_train, target_train)
    predictions_valid = model_lr_new.predict(features_valid)
    accuracy_valid = accuracy_score(target_valid, predictions_valid)
    print('Качество предсказаний модели логистической регрессии с алгоритмом оптимизации {} => {:.2%}'.format(alg, accuracy_valid))

Качество предсказаний модели логистической регрессии с алгоритмом оптимизации newton-cg => 76.83%
Качество предсказаний модели логистической регрессии с алгоритмом оптимизации liblinear => 70.92%
Качество предсказаний модели логистической регрессии с алгоритмом оптимизации sag => 69.98%
Качество предсказаний модели логистической регрессии с алгоритмом оптимизации saga => 69.67%


/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:426: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Качество предсказаний модели, использующей алгоритм логистической регрессии с алгоритмом оптимизации 'newton-cg', выше на 5%, чем со стандартным алгоритмом(‘lbfgs’). Переобучим модель с новым гиперпараметром:

In [21]:
model_lr = LogisticRegression(random_state=random_st, solver='newton-cg')

#обучим модель
model_lr.fit(features_train, target_train);

/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:426: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


### Вывод:

1. На валидационной выборке качество моделей дерева решения и случайного леса удовлетворяют условиям задания - не менее 75% (но не известно, удовлетворяют ли на тестовой выборке)
2. После настройки гиперпараметров и проверки качества предсказаний на валидационной выборке все три модели показывают лучшее качество, а именно accuracy равно:
  - для DecisionTreeClassifier(max_depth=4, random_state=random_st) 81.18%;
  - для RandomForestClassifier(n_estimators=3, max_depth=5, random_state=random_st) 81.8%;
  - для LogisticRegression(random_state=random_st, solver='newton-cg') 76.83%.

## 4. Проверьте модель на тестовой выборке

Сделаем предсказания на тестовой выборке и получим значение качества этих предсказаний:

In [22]:
accuracy_dtc_test = get_accuracy(model_dtc, features_test, target_test)
accuracy_rfc_test = get_accuracy(model_rfc, features_test, target_test)
accuracy_lr_test = get_accuracy(model_lr, features_test, target_test)

In [23]:
print_accuracy(accuracy_dtc_test, accuracy_rfc_test, accuracy_lr_test)

Качество предсказаний модели DecisionTreeClassifier => 78.07%
Качество предсказаний модели RandomForestClassifier => 78.85%
Качество предсказаний модели LogisticRegression => 74.03%


### Вывод:

1. Качество предсказаний на тестовой выборке для дерева решений и случайного леса чуть хуже, чем на валидационной, но, тем не менее, удовлетворяет поставленной задаче и не менее 75%.
2. Модель логистической регрессии не подходит для решения поставленной задачи, либо требуется дальнейшая настройка гиперпараметров.

## 5. (бонус) Проверьте модели на адекватность

Для того, чтобы оценить адекватность модели нужно сравнить качество предсказаний выбранной модели с качеством модели, которая предсказывает лишь один класс. Предположим, 0 для тарифа. Напишем свой классификатор и проверим его качество предсказаний по сравнению с выбранными моделями.

In [24]:
from sklearn.base import BaseEstimator # импортируем класс базовой оценки

Cоздадим класс с методами fit и predict, который будет предсказывать только 0. При тренеровке модели не будет ничего происходить, а метод predict вернет нулевую матрицу.

In [32]:
class Not1Classifier(BaseEstimator):   # наследуем внутреннюю реализацию классификатора от BaseEstimator
    
    def fit(self, features, target):
        pass
    
    def predict(self, features):
        return np.zeros([len(features), 1])

Обучим модель классификатора:

In [33]:
model_not1 = Not1Classifier() # создадим объект классификатора
model_not1.fit(features_train, target_train)  # тренеруем модель на тренеровочных данных

In [34]:
# посмотрим на accuracy предсказаний такой модели
accuracy_not1 = get_accuracy(model_not1, features_test, target_test)
print('Качество предсказаний модели классификатора, предсказывающей только нули для тарифа => {:.2%}'.format(accuracy_not1))

Качество предсказаний модели классификатора, предсказывающей только нули для тарифа => 69.36%


### Вывод:

Как видно из реализации представленного классификатора, предсказывающего только нули, выбранные и обученные модели дают более точные предсказания (более точные на 5-9 %). Можно предположить, что наши модели адекватны в части предсказания тарифов, но разница при предсказании тарифа smart не так уж и велика (но всё относительно).